In [ ]:
pip install opencv-python-headless

Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
import numpy as np

In [4]:
!pip install imutils

  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for imutils: filename=imutils-0.5.4-py3-none-any.whl size=25847 sha256=893ff61859d724aec0174e1d924d816bcef13663dfaca17588b11eef799fc810
  Stored in directory: c:\users\91970\appdata\local\pip\cache\wheels\4b\a5\2d\4a070a801d3a3d93f033d3ee9728f470f514826e89952df3ea
Successfully built imutils


In [12]:
import matplotlib.pyplot as plt

# import the necessary packages
from collections import deque
import numpy as np
import cv2 as cv
import time
import math
import scipy.ndimage
import os

# construct the argument parse and parse the arguments
font = cv.FONT_HERSHEY_SIMPLEX

def orientated_non_max_suppression(mag, ang):
    ang_quant = np.round(ang / (np.pi/4)) % 4
    winE = np.array([[0, 0, 0], [1, 1, 1], [0, 0, 0]])
    winSE = np.array([[1, 0, 0], [0, 1, 0], [0, 0, 1]])
    winS = np.array([[0, 1, 0], [0, 1, 0], [0, 1, 0]])
    winSW = np.array([[0, 0, 1], [0, 1, 0], [1, 0, 0]])
    magE = non_max_suppression(mag, winE)
    magSE = non_max_suppression(mag, winSE)
    magS = non_max_suppression(mag, winS)
    magSW = non_max_suppression(mag, winSW)
    mag[ang_quant == 0] = magE[ang_quant == 0]
    mag[ang_quant == 1] = magSE[ang_quant == 1]
    mag[ang_quant == 2] = magS[ang_quant == 2]
    mag[ang_quant == 3] = magSW[ang_quant == 3]
    return mag

def non_max_suppression(data, win):
    data_max = scipy.ndimage.maximum_filter(
        data, footprint=win, mode='constant')
    data_max[data != data_max] = 0
    return data_max

# Directly set the input and output directories (replace with your paths)
input_dir = 'Downloads/Railway Track fault Detection Updated/Train/Defective'  # Modify as per your dataset path
output_dir = 'Downloads/Railway Track fault Detection Updated/Train/output'  # Modify as needed

# Create output directory if it doesn't exist
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Process directories (Train, Test, Validation)

for data_type in ['Train', 'Test', 'Validation']:
    data_dir = os.path.join(input_dir, data_type)
    for category in ['Defective', 'Non defective']:
        category_dir = os.path.join(data_dir, category)
        output_category_dir = os.path.join(output_dir, data_type, category)

        # Create output subdirectories if not exist
        if not os.path.exists(output_category_dir):
            os.makedirs(output_category_dir)

        # Get all image paths in the category directory
        image_paths = sorted([os.path.join(category_dir, f) for f in os.listdir(category_dir) if os.path.isfile(os.path.join(category_dir, f))])

        # Open text file for writing results
        with open(os.path.join(output_category_dir, 'results.txt'), 'w') as file:
            for image_path in image_paths:
                # load the image
                image_name = os.path.basename(image_path)
                image = cv.imread(image_path)
                frame = imutils.resize(image, width=800)
                gray_image = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)
                img = frame.copy()

                fudgefactor = 1.5
                sigma = 50 # for Gaussian Kernel
                kernel = 2*math.ceil(2*sigma)+1 # Kernel size

                gray_image = gray_image/255.0
                blur = cv.GaussianBlur(gray_image, (kernel, kernel), sigma)
                gray_image = cv.subtract(gray_image, blur)

                # compute sobel response
                sobelx = cv.Sobel(gray_image, cv.CV_64F, 1, 0, ksize=3)
                sobely = cv.Sobel(gray_image, cv.CV_64F, 0, 1, ksize=3)

                mag = np.hypot(sobelx, sobely)
                ang = np.arctan2(sobely, sobelx)

                # threshold
                threshold = 4 * fudgefactor * np.mean(mag)
                mag[mag < threshold] = 0

                # non-maximal suppression
                mag = orientated_non_max_suppression(mag, ang)

                # create mask
                mag[mag > 0] = 255
                mag = mag.astype(np.uint8)

                kernel = np.ones((5, 5), np.uint8)
                result = cv.morphologyEx(mag, cv.MORPH_CLOSE, kernel)
                num_white = np.sum(result == 255)
                num_black = np.sum(result == 0)
                ratio = (num_white/num_black)*100

                if ratio > 0.7:
                    result_text = "Cracked"
                    cv.putText(img, 'Cracked', (0, 30),
                                font, 0.8, (0, 0, 255), 2, cv.LINE_AA)
                else:
                    result_text = "Not Cracked"
                    cv.putText(img, 'Not Cracked', (0, 30),
                                font, 0.8, (0, 255, 0), 2, cv.LINE_AA)

                # Write results to text file
                file.write(f"Image: {image_name}, Outcome: {result_text}, Ratio: {ratio}\n")

                # Save modified image to output directory
                cv.imwrite(os.path.join(output_category_dir, image_name), img)

                # Display the images using matplotlib
                plt.figure(figsize=(10,5))
                plt.subplot(1, 2, 1)
                plt.imshow(cv.cvtColor(img, cv.COLOR_BGR2RGB))
                plt.title('Image')
                plt.axis('off')

                plt.subplot(1, 2, 2)
                plt.imshow(result, cmap='gray')
                plt.title('Result')
                plt.axis('off')

                plt.show()

FileNotFoundError: [WinError 3] The system cannot find the path specified: 'Downloads/Railway Track fault Detection Updated/Train/Defective\\Train\\Defective'

In [ ]:
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, accuracy_score
import numpy as np
import imutils
import cv2
import os
import math  # Make sure to import math if it's not already imported

# Variables for classification tracking
true_labels = []
predicted_labels = []

# Directly set the input and output directories
input_dir = '/kaggle/input/railway-track-fault-detection/Railway Track fault Detection Updated'
output_dir = '/kaggle/working/output'

# Create output directory if it doesn't exist
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Process directories (Train, Test, Validation)
for data_type in ['Train', 'Test', 'Validation']:
    data_dir = os.path.join(input_dir, data_type)
    for category in ['Defective', 'Non defective']:
        category_dir = os.path.join(data_dir, category)
        output_category_dir = os.path.join(output_dir, data_type, category)

        # Create output subdirectories if not exist
        if not os.path.exists(output_category_dir):
            os.makedirs(output_category_dir)

        # Get all image paths in the category directory
        image_paths = sorted([os.path.join(category_dir, f) for f in os.listdir(category_dir) if os.path.isfile(os.path.join(category_dir, f))])

        # Open text file for writing results
        with open(os.path.join(output_category_dir, 'results.txt'), 'w') as file:
            for image_path in image_paths:
                # load the image
                image_name = os.path.basename(image_path)
                image = cv2.imread(image_path)
                frame = imutils.resize(image, width=800)
                gray_image = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
                img = frame.copy()

                fudgefactor = 1.5 
                sigma = 50
                kernel = 2 * math.ceil(2 * sigma) + 1

                gray_image = gray_image / 255.0
                blur = cv2.GaussianBlur(gray_image, (kernel, kernel), sigma)
                gray_image = cv2.subtract(gray_image, blur)

                # compute sobel response
                sobelx = cv2.Sobel(gray_image, cv2.CV_64F, 1, 0, ksize=3)
                sobely = cv2.Sobel(gray_image, cv2.CV_64F, 0, 1, ksize=3)

                mag = np.hypot(sobelx, sobely)
                ang = np.arctan2(sobely, sobelx)

                # threshold
                threshold = 4 * fudgefactor * np.mean(mag)
                mag[mag < threshold] = 0

                # non-maximal suppression
                mag = orientated_non_max_suppression(mag, ang)

                # create mask
                mag[mag > 0] = 255
                mag = mag.astype(np.uint8)

                kernel = np.ones((5, 5), np.uint8)
                result = cv2.morphologyEx(mag, cv2.MORPH_CLOSE, kernel)
                num_white = np.sum(result == 255)
                num_black = np.sum(result == 0)
                ratio = (num_white / num_black) * 100

                if ratio > 0.7:
                    result_text = "Cracked"
                    predicted_labels.append("Defective")
                    cv2.putText(img, 'Cracked', (0, 30), font, 0.8, (0, 0, 255), 2, cv2.LINE_AA)
                else:
                    result_text = "Not Cracked"
                    predicted_labels.append("Non defective")
                    cv2.putText(img, 'Not Cracked', (0, 30), font, 0.8, (0, 255, 0), 2, cv2.LINE_AA)

                # Add true label based on directory
                if "Defective" in category:
                    true_labels.append("Defective")
                else:
                    true_labels.append("Non defective")

                # Write results to text file
                file.write(f"Image: {image_name}, Outcome: {result_text}, Ratio: {ratio}\n")

                # Save modified image to output directory
                cv2.imwrite(os.path.join(output_category_dir, image_name), img)

# Calculate and print accuracy and confusion matrix
accuracy = accuracy_score(true_labels, predicted_labels)
conf_matrix = confusion_matrix(true_labels, predicted_labels)

print(f"Classification Accuracy: {accuracy * 100:.2f}%")
print("Confusion Matrix:")
print(conf_matrix)

# Plot confusion matrix
plt.figure(figsize=(6, 6))
plt.imshow(conf_matrix, interpolation='nearest', cmap=plt.cm.Blues)
plt.title('Confusion Matrix')
plt.colorbar()
plt.xticks(np.arange(2), ['Defective', 'Non defective'], rotation=45)
plt.yticks(np.arange(2), ['Defective', 'Non defective'])
plt.ylabel('True label')
plt.xlabel('Predicted label')

# Annotate confusion matrix with counts
thresh = conf_matrix.max() / 2.  # Threshold for text color
for i in range(conf_matrix.shape[0]):
    for j in range(conf_matrix.shape[1]):
        plt.text(j, i, format(conf_matrix[i, j], 'd'), 
                 ha="center", va="center",
                 color="white" if conf_matrix[i, j] > thresh else "black")

plt.show()


In [ ]:
import matplotlib.pyplot as plt
import os
import cv2

# Set the output directory (the one where you saved the processed images)
output_dir = '/kaggle/working/output'

# Loop through each subdirectory (Train, Test, Validation) and each category (Defective, Non defective)
for data_type in ['Train','Test', 'Validation']:
    data_type_dir = os.path.join(output_dir, data_type)
    for category in ['Defective', 'Non defective']:
        category_dir = os.path.join(data_type_dir, category)

        # Get all image paths in the category directory
        image_paths = sorted([os.path.join(category_dir, f) for f in os.listdir(category_dir) if os.path.isfile(os.path.join(category_dir, f))])

        # Display each image
        for image_path in image_paths:
            # Load the image
            image = cv2.imread(image_path)
            # Convert the image from BGR (OpenCV format) to RGB (Matplotlib format)
            image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

            # Display the image
            plt.figure(figsize=(5, 5))
            plt.imshow(image_rgb)
            plt.title(os.path.basename(image_path))
            plt.axis('off')
            plt.show()
